In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf
import copy

/tmp/ipykernel_5997/2034768389.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import sys

path_common=os.path.dirname(os.path.abspath(''))+"/common/";
sys.path.append(path_common) #Adds common path to import the python files
print("Current working directory:", os.getcwd())

Current working directory: /home/slagarp/exjobb/ExampleCode/scripts/tutorial


In [3]:
sys.path

['/home/slagarp/exjobb/ExampleCode/scripts/tutorial',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/slagarp/.local/lib/python3.10/site-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/slagarp/exjobb/ExampleCode/scripts/common/']

# Dataset Generation
The functions to manipulate the dataset allow for

In [3]:
from DatasetFuncs import allDataset_loader,dataset_split,show_porcentages,show_partition_nanopores

data_folder="../../ext/QuipuData/"; #path where the datasets are/will be stored
allDatasets=allDataset_loader(data_folder) #Generates the processed df and saves it. If it is already in memory
#then it loads it. One can specify cut=False to obtain the uncut traces (if not they are cut at 700 samples as was used in quipunet)

trainSet,testSet=dataset_split(allDatasets)#Divides in train and test, in a way that the test set is within a range of
# % of samples, and that nanopores used on train set are not used in train for the same barcode.
#NOTE: This is to generate random test sets, if you want to use the ones from quipu should be coded!
samples_perc=show_porcentages(trainSet,testSet) #Shows number of samples for train and test set (and then percentages), for each barcode

     Train  Test
000   5769   315
001   8167   838
010   2418   101
011  14632  1977
100    919    83
101   7551   427
110   6731   606
111   6979   665
         Train       Test
000  94.822485   5.177515
001  90.694059   9.305941
010  95.990472   4.009528
011  88.096815  11.903185
100  91.716567   8.283433
101  94.647781   5.352219
110  91.740493   8.259507
111  91.300366   8.699634


/home/slagarp/exjobb/ExampleCode/scripts/common/DatasetFuncs.py:137: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '94.8224852071006' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  samples_perc.iloc[i,0]=100*samples_dist.iloc[i,0]/(samples_dist.iloc[i,0]+samples_dist.iloc[i,1]);
/home/slagarp/exjobb/ExampleCode/scripts/common/DatasetFuncs.py:138: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.177514792899408' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  samples_perc.iloc[i,1]=100*samples_dist.iloc[i,1]/(samples_dist.iloc[i,0]+samples_dist.iloc[i,1]);


In [4]:
#With this code we show which is the nanopores used for this train and test dataset
print(show_partition_nanopores(trainSet))
print(show_partition_nanopores(testSet))

   barcode  nanopore  size
0      000         7  2172
1      000         8  3498
2      000      1017    99
3      001         8  1737
4      001        26   502
5      001        27  3583
6      001        28   923
7      001        29  1255
8      001      1053   167
9      010         7  1021
10     010         9   629
11     010        10   768
12     011         6  1332
13     011         7   702
14     011        12   577
15     011        31   899
16     011        33  2192
17     011        35   483
18     011        36  1780
19     011        37  1583
20     011        38  1229
21     011        39  1735
22     011        40  2120
23     100         7   288
24     100        13   631
25     101         8  1240
26     101        26  1764
27     101        27   924
28     101        29  3494
29     101      1662   129
30     110         7  1161
31     110         8  2702
32     110        13  1309
33     110        14  1559
34     111         7   735
35     111         8  2866
3

# Testing reproducibility

With this function we can run the training of quipu how it was done in their notebook (with n epochs=60)

In [14]:
from ModelTrainer import ModelTrainer
from ModelFuncs import get_quipu_model
mt=ModelTrainer()
model=get_quipu_model();
mt.quipu_def_train(model,n_epochs=2)

=== Epoch: 0 ===
  prep time: 1.4 sec   train time: 52.8 sec
  loss: 1.985   acc: 0.1952   val_acc: 0.2167
=== Epoch: 1 ===
  prep time: 1.4 sec   train time: 51.2 sec
  loss: 1.872   acc: 0.2248   val_acc: 0.2321
       [ loss , accuracy ]
Train: [1.8756051063537598, 0.21382546424865723]
Validation  : [1.8405187129974365, 0.23213452100753784]
Test : [2.393502712249756, 0.11225364357233047]


(0.21382546424865723, 0.23213452100753784, 0.11225364357233047)

This code replicates the training as it is done in the github, but for a random test dataset. This is just to show, but the training I recommend doing is the one which will be shown in the next cell

## Example train

In [5]:
from ModelTrainer import ModelTrainer
from ModelFuncs import create_fcn_model #get_resnet_model
mt=ModelTrainer(n_epochs_max=10); #use_weights option weights classes instead of oversampling. track_losses is used to save each result for every cross-validation
#(model, modelinfo)=get_resnet_model();
model = create_fcn_model((None, 1), 8);
mt.train_es(model)

2024-02-23 18:33:42.240727: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 18:33:42.258591: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 18:33:42.258602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 18:33:42.259095: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 18:33:42.262239: I tensorflow/core/platform/cpu_feature_guar

=== Epoch: 0 ===


2024-02-23 18:33:48.868385: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-23 18:33:48.935994: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-23 18:33:49.100408: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-23 18:33:49.837107: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fd82ce83710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-23 18:33:49.837131: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-02-23 18:33:49.840059: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708709629.894201    6142 device_compiler.

782/782 [==============================] - 67s 81ms/step - loss: 1.4420 - accuracy: 0.5117
Validation ds:
16/16 [==============================] - 4s 165ms/step - loss: 1.8999 - accuracy: 0.4225
  prep time: 2.7 sec   train time: 71.5 sec
=== Epoch: 1 ===
141/782 [====>.........................] - ETA: 53s - loss: 1.2020 - accuracy: 0.5951